In [2]:
####### 데이터셋 불러오고 서브셋 만들기 (vit, swin과 동일)
from itertools import chain
from collections import defaultdict
from torch.utils.data import Subset
from torchvision import datasets


def subset_sampler(dataset, classes, max_len): 
    target_idx = defaultdict(list)
    for idx, label in enumerate(dataset.train_labels): 
        target_idx[int(label)].append(idx)

    indices = list(
        chain.from_iterable(
            [target_idx[idx][:max_len] for idx in range(len(classes))]
        )
    )
    return Subset(dataset, indices)


train_dataset = datasets.FashionMNIST(root="../datasets", download=True, train=True)
test_dataset  = datasets.FashionMNIST(root="../datasets", download=True, train=False)

classes      = train_dataset.classes
class_to_idx = train_dataset.class_to_idx

subset_train_dataset = subset_sampler(
    dataset = train_dataset, classes = train_dataset.classes, max_len = 1000
)
subset_test_dataset = subset_sampler(
    dataset = test_dataset, classes = test_dataset.classes, max_len = 100
)

/opt/homebrew/lib/python3.11/site-packages/torchvision/datasets/mnist.py:65: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")


In [10]:
####### 10.20 CvT모델 이미지 데이터 전처리
import torch
from torchvision import transforms
from transformers import AutoImageProcessor


image_processor = AutoImageProcessor.from_pretrained(
    pretrained_model_name_or_path = "microsoft/cvt-21"
)

transform = transforms.Compose(
    [
        # 모델 학습을 위해 PIL.image > Tensor
        transforms.ToTensor(),
        # 크기 조정
        transforms.Resize(
            size=(
                # 기존에는 224*224 강제조정이라면 여기선 비율유지
                # 예를 들어, 원본 이미지의 크기가 800x600(가로x세로)일 때, shortest_edge를 224로 설정하면 이미지의 세로 길이가 224이 되고, 가로 길이는 원본 비율을 유지하여 224 * (800/600) = 299이 된다. 결과적으로 이미지의 크기는 299x224이 된다.
                image_processor.size["shortest_edge"],  # shortest_edge: 이미지의 너비나 높이 중 더 작은 값 (224)
                image_processor.size["shortest_edge"]
            )
        ),
        # 단일 채널을 복제해 다중 채널 이미지로 변환
            # FashinMNIST의 x는 흑백이미지 [1,H,W]
            # 텐서 3개로 복사하여 0번차원 기준으로 cat(연결)
            #결과: [3,H,W]
        transforms.Lambda(lambda x: torch.cat([x, x, x], 0)),
        # 정규화
        transforms.Normalize(
            mean = image_processor.image_mean,
            std  = image_processor.image_std
        )
    ]
)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [4]:
####### 모델 구조에 맞는 형태로 변환 데이터로더 적용
from torch.utils.data import DataLoader


# Tensor 형식이 아닌 딕셔너리 형식의 데이터를 입력으로 받음
# 모델의 입력: {"pixel_values":pixel_values, "labels":labels}
def collator(data, transform):
    images, labels = zip(*data)
    # pixel_values = (배치크기, 채널수, 이미지높이, 이미지너비)
    pixel_values = torch.stack([transform(image) for image in images])
    # labels = [클래스 색인값]
    labels       = torch.tensor([label for label in labels])
    return {"pixel_values": pixel_values, "labels": labels}


train_dataloader = DataLoader(
    subset_train_dataset,
    batch_size = 32,
    shuffle    = True,
    collate_fn = lambda x: collator(x, transform),
    drop_last  = True
)
valid_dataloader = DataLoader(
    subset_test_dataset,
    batch_size = 4,
    shuffle    = True,
    collate_fn = lambda x: collator(x, transform),
    drop_last  = True
)

In [5]:
####### 10.21 사전 학습된 CvT 모델
from transformers import CvtForImageClassification


model = CvtForImageClassification.from_pretrained(
    # 사전 학습된 모델은 앞의 이미지 프로세서 클래스에서 사용한 모델과 동일
    pretrained_model_name_or_path = "microsoft/cvt-21",
    # 아래의 3개 매개변수를 통해 현재 데이터세트에 적합한 구조로 모델을 미세조정
    num_labels                    = len(train_dataset.classes),
    id2label                      = {idx: label for label, idx in train_dataset.class_to_idx.items()},
    label2id                      = train_dataset.class_to_idx,
    ignore_mismatched_sizes       = True
)

for main_name, main_module in model.named_children(): 
    print(main_name)
    for sub_name, sub_module in main_module.named_children(): 
        print("└", sub_name)
        for ssub_name, ssub_module in sub_module.named_children(): 
            print("   └", ssub_name)
            for sssub_name, sssub_module in ssub_module.named_children(): 
                print("     └", sssub_name)

/opt/homebrew/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of CvtForImageClassification were not initialized from the model checkpoint at microsoft/cvt-21 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 384]) in the checkpoint and torch.Size([10, 384]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([10]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


cvt
└ encoder
   └ stages
     └ 0
     └ 1
     └ 2
layernorm
classifier


스윈 트랜스포머와의 차이점  
1. classifier 이전에 pooler 계층 존재 X
2. 스테이지별 이미지 크기와 임베딩 차원
    - 이미지 사이즈는 스테이지별로 4배, 8배, 16배 감소됨
    - 이미지 사이즈와 반비례하게 임베딩 차원이 커짐

In [6]:
####### 10.22 CvT 모델의 스테이지 구조
# CvT Stage = CvT Embeddings(패치 단위로 잘라 임베딩) + CvT Layer()
stages = model.cvt.encoder.stages
print(stages[0])

'''
CvT Embedding
- 224*224 > 56*56(소수점버림) => [N,64,56,56]
CvT Layer
- 매개변수가 축소된 어텐션 임베딩 계산 위해 key, value 임베딩 간격을 1보다 큰 2로 설정해 차원 축소
- 3136(56*56)개의 이미지 패치에 대한 셀프 어텐션 수행
'''

CvtStage(
  (embedding): CvtEmbeddings(
    (convolution_embeddings): CvtConvEmbeddings(
      (projection): Conv2d(3, 64, kernel_size=(7, 7), stride=(4, 4), padding=(2, 2))
      (normalization): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    )
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (layers): Sequential(
    (0): CvtLayer(
      (attention): CvtAttention(
        (attention): CvtSelfAttention(
          (convolution_projection_query): CvtSelfAttentionProjection(
            (convolution_projection): CvtSelfAttentionConvProjection(
              (convolution): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
              (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            )
            (linear_projection): CvtSelfAttentionLinearProjection()
          )
          (convolution_projection_key): CvtSelfAttentionProjection(
            (convolution_projection): CvtSelfAtte

'\nCvT Embedding\n- 224*224 > 56*56(소수점버림) => [N,64,56,56]\nCvT Layer\n- 매개변수가 축소된 어텐션 임베딩 계산 위해 key, value 임베딩 간격을 1보다 큰 2로 설정해 차원 축소\n- 3136(56*56)개의 이미지 패치에 대한 셀프 어텐션 수행\n'

In [8]:
####### 10.23 셀프 어텐션 적용
batch = next(iter(train_dataloader))
print("이미지 차원 :", batch["pixel_values"].shape)

patch_emb_output = stages[0].embedding(batch["pixel_values"])
print("패치 임베딩 차원 :", patch_emb_output.shape)

batch_size, num_channels, height, width = patch_emb_output.shape
hidden_state = patch_emb_output.view(batch_size, num_channels, height * width).permute(0, 2, 1)
print("셀프 어텐션 입력 차원 :", hidden_state.shape)

# self-attention 수행 (입력 차원과 출력 차원 형태 동일)
attention_output = stages[0].layers[0].attention.attention(hidden_state, height, width)
print("셀프 어텐션 출력 차원 :", attention_output.shape)

이미지 차원 : torch.Size([32, 3, 224, 224])
패치 임베딩 차원 : torch.Size([32, 64, 56, 56])
셀프 어텐션 입력 차원 : torch.Size([32, 3136, 64])
셀프 어텐션 출력 차원 : torch.Size([32, 3136, 64])
